# Mahtab Nejati
# 98209434
# Problem 03

In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PoissonRegressor,TweedieRegressor,GammaRegressor,LinearRegression
from sklearn.metrics import mean_squared_error

## 1. Loading in data

In [2]:
train_df = pd.read_csv('./q3/train.csv',index_col='Id')
test_df = pd.read_csv('./q3/test.csv',index_col='Id')
test = test_df.copy()

## 2&3. Cleaning data & preprocess

## NOTICE
### Since the test data does not include the results, I have decided to split the train data into two chuncks, train and validation. This helps to evaluate the model at the end.

In [3]:
train_df , validation_df = train_test_split(train_df,test_size = 0.2)
train = train_df.copy()
validation = validation_df.copy()

## Normalize

## Notice 
### I have made the assumption that at the time of cleaning the data, only the training data is available. That's why I have saved the mean and range value of each feature so that I would normalize the validatoin and test data with the same parameters.

## Relative Numerical Features Normalization: according to the description file

### No Nan
### Only Normalizing MSSubClass

In [4]:
relative_numerics = ['MSSubClass',
                     'OverallQual',
                     'OverallCond',
                     'MoSold']

train['MSSubClass'] = ((train['MSSubClass']/10)-1.5)*2
validation['MSSubClass'] = ((validation['MSSubClass']/10)-1.5)*2
test['MSSubClass'] = ((test['MSSubClass']/10)-1.5)*2

## Real-Number Numerical Features Normalization: according to the description file

### Fixing NaNs

In [5]:
real_numerics = ['LotFrontage',
                 'LotArea',
                 'YearBuilt',
                 'YearRemodAdd',
                 'MasVnrArea',
                 'BsmtFinSF1',
                 'BsmtFinSF2',
                 'BsmtUnfSF',
                 'TotalBsmtSF',
                 '1stFlrSF',
                 '2ndFlrSF',
                 'LowQualFinSF',
                 'GrLivArea',
                 'BsmtFullBath',
                 'BsmtHalfBath',
                 'FullBath',
                 'HalfBath',
                 'BedroomAbvGr',
                 'KitchenAbvGr',
                 'TotRmsAbvGrd',
                 'Fireplaces',
                 'GarageYrBlt',
                 'GarageCars',
                 'GarageArea',
                 'WoodDeckSF',
                 'OpenPorchSF',
                 'EnclosedPorch',
                 '3SsnPorch',
                 'ScreenPorch',
                 'PoolArea',
                 'MiscVal',
                 'YrSold']


isNan_dict = {'train':[],
              'validation':[],
              'test':[]}

for col in real_numerics:
    if(train_df[col].isnull().values.any()):
        isNan_dict['train'].append(col)
    if(validation_df[col].isnull().values.any()):
        isNan_dict['validation'].append(col)
    if(test_df[col].isnull().values.any()):
        isNan_dict['test'].append(col)

isNan = list(set(isNan_dict['train']+isNan_dict['validation']+isNan_dict['test']))
isNan.sort()
print('\n\t### NaN-Containing Columns:\n')
print('\t'.join(isNan))


	### NaN-Containing Columns:

BsmtFinSF1	BsmtFinSF2	BsmtFullBath	BsmtHalfBath	BsmtUnfSF	GarageArea	GarageCars	GarageYrBlt	LotFrontage	MasVnrArea	TotalBsmtSF


### BsmtFinSF1	BsmtFinSF2	BsmtFullBath	BsmtHalfBath	BsmtUnfSF	TotalBsmtSF: 
#### NaN could mean there is no basement or the basement has no bathroom in the building. So fill with 0.

In [6]:
fillnan_cols = ['BsmtFinSF1','BsmtFinSF2','BsmtFullBath','BsmtHalfBath','BsmtUnfSF','TotalBsmtSF']
for col in fillnan_cols:
    train[col].fillna(0,inplace=True)
    validation[col].fillna(0,inplace=True)
    test[col].fillna(0,inplace=True)

### GarageArea	GarageCars	GarageYrBlt: 
#### NaN could mean there is no garage in the building. So fill with 0.

In [7]:
fillnan_cols = ['GarageArea','GarageCars','GarageYrBlt']
for col in fillnan_cols:
    train[col].fillna(0,inplace=True)
    validation[col].fillna(0,inplace=True)
    test[col].fillna(0,inplace=True)

### LotFrontage:
#### NaN could mean there is no lot in the building. So fill with 0.

In [8]:
train['LotFrontage'].fillna(0,inplace=True)
validation['LotFrontage'].fillna(0,inplace=True)
test['LotFrontage'].fillna(0,inplace=True)

### MasVnrArea:
#### NaN could mean there is no mansor veneer wall in the building. So fill with 0.

In [9]:
train['MasVnrArea'].fillna(0,inplace=True)
validation['MasVnrArea'].fillna(0,inplace=True)
test['MasVnrArea'].fillna(0,inplace=True)

### Mean and Rnge Normalization
#### In order to keep all feature values relatively in the same range, I normalize the range up to -10 to 19 (categoricals go up to 10)

In [10]:
for col in real_numerics:
        
    mean_val = train_df[col].mean()
    range_val = train_df[col].max()-train_df[col].min()

    train[col] = ((train[col]-mean_val)/range_val)*10
    validation[col] = ((validation[col]-mean_val)/range_val)*10
    test[col] = ((test[col]-mean_val)/range_val)*10

## Categorical Features Normalization: according to the description file

### Meaningful NaNs: according to the description file. No need to fix 

In [11]:
meaningful_nans = ['Alley',
                   'BsmtQual',
                   'BsmtCond',
                   'BsmtExposure',
                   'BsmtFinType1',
                   'BsmtFinType2',
                   'FireplaceQu',
                   'GarageType',
                   'GarageFinish',
                   'GarageQual',
                   'GarageCond',
                   'PoolQC',
                   'Fence',
                   'MiscFeature']

for col in meaningful_nans:
    train[col].fillna('NA',inplace=True)
    validation[col].fillna('NA',inplace=True)
    test[col].fillna('NA',inplace=True)

### one-hot encoding for categorical features that cannot be relatively enumerated (e,g. the LandSlope of , moderate, and low can be enumerated to 3,2,1. but it's not the same with MSZoning since non of the categories has a relative value compared with the others.

In [12]:
categories = ['MSZoning',
              'LotConfig',
              'Neighborhood',
              'BldgType',
              'HouseStyle',
              'RoofStyle',
              'RoofMatl',
              'MasVnrType',
              'Foundation',
              'Heating',
              'Electrical',
              'Functional',
              'GarageType',
              'MiscFeature',
              'SaleType',
              'SaleCondition']

for col in categories:
    one_hot = pd.get_dummies(train_df[col],prefix=col)
    try:
        one_hot.drop(col+'_NA',axis=1)
    except KeyError:
        pass

    train = train.drop(col, axis=1)
    train = train.join(one_hot)

    for dummy in one_hot.columns:
        validation[dummy] = int(dummy in validation[col])
        test[dummy] = int(dummy in test[col])

    validation = validation.drop(col, axis=1)
    test = test.drop(col, axis=1)

### Enumerating relative Non-numeric values.

In [13]:
relatives = ['Street',
             'Alley',
             'LotShape',
             'LandContour',
             'Utilities',
             'LandSlope',
             'ExterQual',
             'ExterCond',
             'BsmtQual',
             'BsmtCond',
             'BsmtExposure',
             'HeatingQC',
             'CentralAir',
             'KitchenQual',
             'FireplaceQu',
             'GarageFinish',
             'GarageQual',
             'GarageCond',
             'PavedDrive',
             'PoolQC',
             'Fence']

### Street, Alley

In [14]:
enum_cols = ['Street','Alley']
enum_dict = {'NA':0,
             'Grvl':1,
             'Pave':2}

for col in enum_cols:
    train.replace({col:enum_dict},inplace=True)
    validation.replace({col:enum_dict},inplace=True)
    test.replace({col:enum_dict},inplace=True)

### LotShape

In [15]:
enum_dict = {'Reg':0,
             'IR1':1,
             'IR2':2,
             'IR3':3}

train.replace({'LotShape':enum_dict},inplace=True)
validation.replace({'LotShape':enum_dict},inplace=True)
test.replace({'LotShape':enum_dict},inplace=True)

### LandContour

In [16]:
enum_dict = {'Lvl':0,
             'Low':1,
             'Bnk':2,
             'HLS':3}

train.replace({'LandContour':enum_dict},inplace=True)
validation.replace({'LandContour':enum_dict},inplace=True)
test.replace({'LandContour':enum_dict},inplace=True)

### Utilities

In [17]:
enum_dict = {'ELO':0,
             'NoSeWa':1,
             'NoSewr':2,
             'AllPub':3}

train.replace({'Utilities':enum_dict},inplace=True)
validation.replace({'Utilities':enum_dict},inplace=True)
test.replace({'Utilities':enum_dict},inplace=True)

### LandSlope

In [18]:
enum_dict = {'Gtl':0,
             'Mod':1,
             'Sev':2}

train.replace({'LandSlope':enum_dict},inplace=True)
validation.replace({'LandSlope':enum_dict},inplace=True)
test.replace({'LandSlope':enum_dict},inplace=True)

### ExterQual, ExterCond, BsmtQual, BsmtCond, HeatingQC, KitchenQual, FireplaceQu, GarageQual, GarageCond, PoolQC

In [19]:
enum_cols = ['ExterQual',
             'ExterCond', 
             'BsmtQual', 
             'BsmtCond', 
             'HeatingQC', 
             'KitchenQual', 
             'FireplaceQu', 
             'GarageQual', 
             'GarageCond', 
             'PoolQC']
enum_dict = {'Ex':5,
             'Gd':4,
             'TA':3,
             'Fa':2,
             'Po':1,
             'NA':0}

for col in enum_cols:
    train.replace({col:enum_dict},inplace=True)
    validation.replace({col:enum_dict},inplace=True)
    test.replace({col:enum_dict},inplace=True)

### BsmtExposure

In [20]:
enum_dict = {'Gd':4,
             'Av':3,
             'Mn':2,
             'No':1,
             'NA':0}

train.replace({'BsmtExposure':enum_dict},inplace=True)
validation.replace({'BsmtExposure':enum_dict},inplace=True)
test.replace({'BsmtExposure':enum_dict},inplace=True)

### CentralAir

In [21]:
enum_dict = {'Y':1,
             'N':0}

train.replace({'CentralAir':enum_dict},inplace=True)
validation.replace({'CentralAir':enum_dict},inplace=True)
test.replace({'CentralAir':enum_dict},inplace=True)

### GarageFinish

In [22]:
enum_dict = {'Fin':3,
             'RFn':2,
             'Unf':1,
             'NA':0}

train.replace({'GarageFinish':enum_dict},inplace=True)
validation.replace({'GarageFinish':enum_dict},inplace=True)
test.replace({'GarageFinish':enum_dict},inplace=True)

### PavedDrive

In [23]:
enum_dict = {'Y':2,
             'P':1,
             'N':0}

train.replace({'PavedDrive':enum_dict},inplace=True)
validation.replace({'PavedDrive':enum_dict},inplace=True)
test.replace({'PavedDrive':enum_dict},inplace=True)

### Fence

In [24]:
enum_dict = {'GdPrv':4,
             'MnPrv':3,
             'GdWo':2,
             'MnWw':1,
             'NA':0}

train.replace({'Fence':enum_dict},inplace=True)
validation.replace({'Fence':enum_dict},inplace=True)
test.replace({'Fence':enum_dict},inplace=True)

## Special Cases
#### Merge pairs into one column and then apply multilable binarization

In [25]:
spec = ['Condition1','Condition2',
        'Exterior1st','Exterior2nd',
        'BsmtFinType1','BsmtFinType2']

mlb = MultiLabelBinarizer()

### Condition1 and Condition1

In [26]:
train['Condition'] = train[['Condition1','Condition2']].apply(lambda row: list(row.values.astype(str)),axis=1)

train = train.drop(['Condition1','Condition2'],axis=1)

one_hot = pd.DataFrame(mlb.fit_transform(train.pop('Condition')),columns=mlb.classes_,index=train.index).add_prefix('Condition_')

train = train.join(one_hot)

for dummy in one_hot.columns:
        validation[dummy] = int(dummy in validation['Condition1'] or dummy in validation['Condition2'])
        test[dummy] = int(dummy in validation['Condition1'] or dummy in validation['Condition2'])

validation = validation.drop(['Condition1','Condition2'], axis=1)
test = test.drop(['Condition1','Condition2'], axis=1)

### Exterior1st and Exterior2nd

In [27]:
train['Exterior'] = train[['Exterior1st','Exterior2nd']].apply(lambda row: list(row.values.astype(str)),axis=1)

train = train.drop(['Exterior1st','Exterior2nd'],axis=1)

one_hot = pd.DataFrame(mlb.fit_transform(train.pop('Exterior')),columns=mlb.classes_,index=train.index).add_prefix('Exterior_')

train = train.join(one_hot)

for dummy in one_hot.columns:
        validation[dummy] = int(dummy in validation['Exterior1st'] or dummy in validation['Exterior2nd'])
        test[dummy] = int(dummy in validation['Exterior1st'] or dummy in validation['Exterior2nd'])

validation = validation.drop(['Exterior1st','Exterior2nd'], axis=1)
test = test.drop(['Exterior1st','Exterior2nd'], axis=1)

### Condition1 and Condition1

In [28]:
train['BsmtFinType'] = train[['BsmtFinType1','BsmtFinType2']].apply(lambda row: list(row.values.astype(str)),axis=1)

train = train.drop(['BsmtFinType1','BsmtFinType2'],axis=1)

one_hot = pd.DataFrame(mlb.fit_transform(train.pop('BsmtFinType')),columns=mlb.classes_,index=train.index).add_prefix('BsmtFinType_')

train = train.join(one_hot)

for dummy in one_hot.columns:
        validation[dummy] = int(dummy in validation['BsmtFinType1'] or dummy in validation['BsmtFinType2'])
        test[dummy] = int(dummy in validation['BsmtFinType1'] or dummy in validation['BsmtFinType2'])

validation = validation.drop(['BsmtFinType1','BsmtFinType2'], axis=1)
test = test.drop(['BsmtFinType1','BsmtFinType2'], axis=1)

### Check for NaN values

In [29]:
train.isnull().values.any()

False

In [30]:
validation.isnull().values.any()

False

In [31]:
test.isnull().values.any()

True

In [32]:
for col in test.columns:
    if test[col].isnull().values.any():
        print(col,end='\t')

Utilities	KitchenQual	

#### Utilities set to electricity only as default (0)

In [33]:
test['Utilities'].fillna(0,inplace=True)

#### KitchenQual set to typical/average as default (3)

In [34]:
test['KitchenQual'].fillna(3,inplace=True)

### Final NaN test

In [35]:
test.isnull().values.any()

False

### SalePrice Normalization
#### normalized to the range 0-100 for convergence reasons

In [36]:
price_max = train_df['SalePrice'].max()

train['NormalizedPrice'] = (train['SalePrice']/price_max)*100
validation['NormalizedPrice'] = (validation['SalePrice']/price_max)*100

### Separate x and y

In [37]:
y_train = train['NormalizedPrice'].copy()
y_train_original = train['SalePrice'].copy()
x_train = train.drop(columns=['SalePrice','NormalizedPrice'])
y_validation = validation['NormalizedPrice'].copy()
y_validation_original = validation['SalePrice'].copy()
x_validation = validation.drop(columns=['SalePrice','NormalizedPrice'])

## 4&5. Make the Model and train it
#### There are three types of GLM implemented in scikit-learn's linear_model module: 
#### PoissonRegressor, TweedieRegressor, and GammaRegressor. 
#### I have experimented all three of them to determine which works best. I also tested out a LinearRegression to see what the results would be like.
## NOTICE: For descriptions on parameters, refer to the report pdf!
#### To get the optimal parameters for each of the models, I have performed a grid search using GridSearchCV from scikit-learn's model_selection module. This search technique is used for exhaustive search over specified parameter values for an estimator to determine the optimal set of parameters. The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid. GridSearchCV tries out all the possible combinations of the specified parameters and returns the chooses the optimal parameters according to the cross-validation results. GridSearchCV implements a “fit” and a “score” method. It also implements “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used. RandomizedSearchCV is another method from scikit-learn's model_selection module that has a similar functionality. The reason I haven't opted for using this method is that in contrast to GridSearchCV, not all parameter values are tried out in this method, but rather a fixed number of parameter settings is sampled from the specified distributions. The number of parameter settings that are tried is given by n_iter but why go the extra mile.

## NOTICE: An interesting fact
### Each time I splitt the data into train and validation, it seems that the distribution varies so much that the results of the GridSearchCV changes drastically. The reason behind this could be the fact that I have used one-hot encoding and increased the number of my features. This results in data sparsity, which might lead to varying distribution over multiple executions. So be mingful that if you run the whole file, results will be different. But the GridSearchCV reveal the same optimal parameters as long as the splitting of data is not altered.

### PoissonRegressor GLM

In [38]:
parameters_psn = {'alpha':[x*0.1 for x in list(range(1,11))],
                  'max_iter':[10000]}
poisson_best = GridSearchCV(PoissonRegressor(), parameters_psn)
poisson_best.fit(x_train, y_train)
best_params_psn = poisson_best.best_params_
poisson = PoissonRegressor(**best_params_psn)
poisson.fit(x_train,y_train)
print(best_params_psn)

{'alpha': 0.9, 'max_iter': 10000}


### TweedieRegressor GLM
#### I experimented with an 'identity' link function and got disasterous results, hence omitted from the parameters list.

In [39]:
parameters_twd = {'power': [0,1,1.5,2,2.5,3],
                  'alpha':[x*0.1 for x in list(range(1,11))],
                  'link':['auto','log'],
                  'max_iter':[10000]}
tweedie_best = GridSearchCV(TweedieRegressor(), parameters_twd)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    tweedie_best.fit(x_train, y_train)
best_params_twd = tweedie_best.best_params_
tweedie = TweedieRegressor(**best_params_twd)
tweedie.fit(x_train,y_train)
print(best_params_twd)

{'alpha': 0.1, 'link': 'auto', 'max_iter': 10000, 'power': 2}


### GammaRegressor GLM

In [40]:
parameters_gmm = {'alpha':[x*0.1 for x in list(range(1,11))],
                  'max_iter':[10000]}
gamma_best = GridSearchCV(GammaRegressor(), parameters_gmm) 
gamma_best.fit(x_train, y_train)
best_params_gmm = gamma_best.best_params_
gamma = GammaRegressor(**best_params_gmm)
gamma.fit(x_train,y_train)
print(best_params_gmm)

{'alpha': 0.1, 'max_iter': 10000}


### LinearRegression

In [41]:
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

## 6. MSE for train and validation data

### PoissonRegressor GLM

In [42]:
train_prediction_psn = poisson.predict(x_train)*price_max/100
train_MSE_psn = mean_squared_error(y_train_original,train_prediction_psn)

validation_prediction_psn = poisson.predict(x_validation)*price_max/100
validation_MSE_psn = mean_squared_error(y_validation_original,validation_prediction_psn)

print('Training data MSE with PoissonRegressor:\t'+str(train_MSE_psn))
print('ValidAaion data MSE with PoissonRegressor:\t'+str(validation_MSE_psn))

Training data MSE with PoissonRegressor:	699118107.9345186
ValidAaion data MSE with PoissonRegressor:	1983945887.2347438


### TweedieRegressor GLM

In [43]:
train_prediction_twd = tweedie.predict(x_train)*price_max/100
train_MSE_twd = mean_squared_error(y_train_original,train_prediction_twd)

validation_prediction_twd = tweedie.predict(x_validation)*price_max/100
validation_MSE_twd = mean_squared_error(y_validation_original,validation_prediction_twd)

print('Training data MSE with TweedieRegressor:\t'+str(train_MSE_twd))
print('Validation data MSE with TweedieRegressor:\t'+str(validation_MSE_twd))

Training data MSE with TweedieRegressor:	1597797452.9000711
Validation data MSE with TweedieRegressor:	1090621229.3533208


### GammaRegressor GLM

In [44]:
train_prediction_gmm = gamma.predict(x_train)*price_max/100
train_MSE_gmm = mean_squared_error(y_train_original,train_prediction_gmm)

validation_prediction_gmm = gamma.predict(x_validation)*price_max/100
validation_MSE_gmm = mean_squared_error(y_validation_original,validation_prediction_gmm)

print('Training data MSE with GammaRegressor:\t\t'+str(train_MSE_gmm))
print('Validation data MSE with GammaRegressor:\t'+str(validation_MSE_gmm))

Training data MSE with GammaRegressor:		1597797452.9000711
Validation data MSE with GammaRegressor:	1090621229.3533208


### LinearRegression

In [46]:
train_prediction_lr = lr.predict(x_train)*price_max/100
train_MSE_lr = mean_squared_error(y_train_original,train_prediction_lr)

validation_prediction_lr = lr.predict(x_validation)*price_max/100
validation_MSE_lr = mean_squared_error(y_validation_original,validation_prediction_lr)

print('Training data MSE with LinearRegression:\t'+str(train_MSE_lr))
print('Validation data MSE with LinearRegression:\t'+str(validation_MSE_lr))

Training data MSE with LinearRegression:	517341865.269234
Validation data MSE with LinearRegression:	4415013005.894116


## 7. Kaggle Results
### Both TweedieRegressor and GammaRegressor reveal exactly the same MSE values which performs slightly better than the PoissonRegressor and LinearRegression. Thus, I chose one of the two models for submision on kaggle. 
### I have used the trained the model once and then retrained the model with all the data from train and validation.
## See the report pdf for results of my submission on kaggle.

In [48]:
prediction_test_splitted = tweedie.predict(test)*price_max/100


x_whole = pd.concat([x_train,x_validation])
y_whole = pd.concat([y_train,y_validation])

tweedie_whole = TweedieRegressor(**best_params_twd)
tweedie_whole.fit(x_whole, y_whole)

prediction_test_whole = tweedie_whole.predict(test)*price_max/100

In [49]:
result_splitted = pd.DataFrame({'Id':test.index,'SalePrice':prediction_test_splitted})
result_whole = pd.DataFrame({'Id':test.index,'SalePrice':prediction_test_whole})

result_splitted.to_csv('q3_result_splitted.csv',index=False)
result_whole.to_csv('q3_result_whole.csv',index=False)